In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv("data/movies.csv")
display(df1.head())
display(df1.info())

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


None

In [3]:
df1['movieId'].nunique()

9742

In [4]:
df2 = pd.read_csv("data/links.csv")
display(df2.head())
display(df2.info())

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


None

In [5]:
df2['movieId'].nunique()

9742

Don't need links.csv because it is just an id from imdb, tmdb and movieId is consistent across all 

In [6]:
df3 = pd.read_csv("data/ratings.csv")
display(df3.head())
display(df3.info())

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


None

In [7]:
df3['movieId'].nunique()

9724

In [8]:
df4 = pd.read_csv("data/tags.csv")
display(df4.head())
display(df4.info())

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


None

In [9]:
df4['movieId'].nunique()

1572

In [10]:
df4['tag'].value_counts()

In Netflix queue     131
atmospheric           36
thought-provoking     24
superhero             24
Disney                23
                    ... 
ethics                 1
Mel Gibson             1
dumb                   1
predictible plot       1
mining                 1
Name: tag, Length: 1589, dtype: int64

In [11]:
movie = df1.merge(df3,on='movieId')

In [12]:
movie.loc[movie['title'] == "Ready Player One"]

,movieId,title,genres,userId,rating,timestamp
99325,140956,Ready Player One,Action|Sci-Fi|Thriller,209,3.5,1524522239
99326,140956,Ready Player One,Action|Sci-Fi|Thriller,305,4.5,1533043577
99327,140956,Ready Player One,Action|Sci-Fi|Thriller,380,3.0,1534176158
99328,140956,Ready Player One,Action|Sci-Fi|Thriller,514,3.0,1533946119


In [13]:
movie.tail()

,movieId,title,genres,userId,rating,timestamp
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,4.0,1537109082
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,3.5,1537109545
100833,193585,Flint (2017),Drama,184,3.5,1537109805
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,1537110021
100835,193609,Andrew Dice Clay: Dice Rules (1991),Comedy,331,4.0,1537157606


In [14]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    100836 non-null  int64  
 1   title      100836 non-null  object 
 2   genres     100836 non-null  object 
 3   userId     100836 non-null  int64  
 4   rating     100836 non-null  float64
 5   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 5.4+ MB


In [15]:
movie['movieId'].nunique()

9724

Find a way to merge tags with nulls 

In [16]:
movie['year'] = movie.title.str.split(" ").str.get(-1)

In [17]:
movie.head()

,movieId,title,genres,userId,rating,timestamp,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,(1995)
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,(1995)
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,(1995)
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,(1995)
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,(1995)


In [18]:
movie['title'] = movie.title.str.rsplit(' ',1).str[0]

In [19]:
movie.head()

,movieId,title,genres,userId,rating,timestamp,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,(1995)
1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,(1995)
2,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,(1995)
3,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,(1995)
4,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,(1995)


In [20]:
movie['year'] = movie['year'].str.replace(r'[()]',"")

In [21]:
movie.head()

,movieId,title,genres,userId,rating,timestamp,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,1995
1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,1995
2,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,1995
3,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,1995
4,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,1995


In [22]:
movie['year'].value_counts()

1995         6143
1994         5296
1999         4535
1996         4509
2000         4268
             ... 
2006–2007       1
1917            1
Animals         1
Watson          1
Paterson        1
Name: year, Length: 120, dtype: int64

In [23]:
movie.loc[movie['year'] == "Baby"]

,movieId,title,genres,userId,rating,timestamp,year
100428,171631,Maria Bamford: Old,(no genres listed),111,1.0,1517440909,Baby


In [24]:
movie['year'].unique()

array(['1995', '1994', '1996', '1976', '1992', '1967', '1993', '1964',
       '1977', '1965', '1982', '1990', '1991', '1989', '1937', '1940',
       '1969', '1981', '1973', '1970', '1955', '1959', '1968', '1988',
       '1997', '1972', '1943', '1952', '1951', '1957', '1961', '1958',
       '1954', '1934', '1944', '1960', '1963', '1942', '1941', '1953',
       '1939', '1950', '1946', '1945', '1938', '1947', '1935', '1936',
       '1956', '1949', '1932', '1975', '1974', '1971', '1979', '1987',
       '1986', '1980', '1978', '1985', '1966', '1962', '1983', '1984',
       '1948', '1933', '1931', '1922', '1998', '1929', '1930', '1927',
       '1928', '1999', '2000', '1926', '1919', '1921', '1925', '1923',
       '2001', '2002', '2003', '1920', '1915', '1924', '2004', '1916',
       '1917', '', '2005', '2006', '1902', '5', '1903', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '1908', 'One', 'Road', 'Watson', 'Animals',
       'P

In [25]:
movie.loc[movie['year'] == 'One']

,movieId,title,genres,userId,rating,timestamp,year
99325,140956,Ready Player,Action|Sci-Fi|Thriller,209,3.5,1524522239,One
99326,140956,Ready Player,Action|Sci-Fi|Thriller,305,4.5,1533043577,One
99327,140956,Ready Player,Action|Sci-Fi|Thriller,380,3.0,1534176158,One
99328,140956,Ready Player,Action|Sci-Fi|Thriller,514,3.0,1533946119,One


In [26]:
movie.loc[movie['year'] == '']

,movieId,title,genres,userId,rating,timestamp,year
78314,27008,From Dusk Till Dawn 2: Texas Blood Money (1999),Comedy|Crime|Horror,599,1.0,1519333508,
86860,58404,Justice League: The New Frontier (2008),Action|Adventure|Animation|Fantasy|Sci-Fi,380,3.0,1495475893,
86861,58404,Justice League: The New Frontier (2008),Action|Adventure|Animation|Fantasy|Sci-Fi,599,2.5,1498517213,
87046,58842,Assembly (Ji jie hao) (2007),Action|Drama|War,339,4.5,1494291470,
94340,94494,96 Minutes (2011),Drama|Thriller,448,2.5,1397303036,
94461,95004,Superman/Doomsday (2007),Action|Animation,380,4.0,1495476116,
94521,95193,Pocahontas II: Journey to a New World (1998),Adventure|Animation|Children,509,2.0,1436000518,
94786,96608,Runaway Brain (1995),Animation|Comedy|Sci-Fi,89,5.0,1520409497,
95536,102084,Justice League: Doom (2012),Action|Animation|Fantasy,380,5.0,1493421861,
95943,104017,3 dev adam (Three Giant Men) (1973),Action|Adventure|Sci-Fi,599,0.5,1519369145,


In [27]:
movie['flag'] = movie.apply(lambda k: 1 if(k['year']=='') else
                           1 if(k['year']=='5') else
                           1 if(k['year']=='One') else
                           1 if(k['year']=='Road') else
                           1 if(k['year']=='Watson') else
                           1 if(k['year']=='Animals') else
                           1 if(k['year']=='Paterson') else
                           1 if(k['year']=='Moonlight') else
                           1 if(k['year']=='OA') else
                           1 if(k['year']=='Cosmos') else
                           1 if(k['year']=='Baby') else
                           1 if(k['year']=='2006-2007') else
                           1 if(k['year']=='2') else
                           1 if(k['year']=='Mirror') else
                           0
                           , axis=1)

In [28]:
movie = movie[movie['flag']!=1]
movie = movie.drop ('flag', axis=1)
movie

,movieId,title,genres,userId,rating,timestamp,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,1995
1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,1995
2,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,1995
3,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,1995
4,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,1995
...,...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy,184,4.0,1537109082,2017
100832,193583,No Game No Life: Zero,Animation|Comedy|Fantasy,184,3.5,1537109545,2017
100833,193585,Flint,Drama,184,3.5,1537109805,2017
100834,193587,Bungo Stray Dogs: Dead Apple,Action|Animation,184,3.5,1537110021,2018


In [29]:
movie['year'].value_counts()

1995         6143
1994         5296
1999         4535
1996         4509
2000         4268
             ... 
1915            1
2006–2007       1
1908            1
1919            1
1917            1
Name: year, Length: 107, dtype: int64

In [30]:
movie.loc[movie['year'] == '2006–2007', 'year'] = '2008'

In [31]:
movie['year'].value_counts()

1995    6143
1994    5296
1999    4535
1996    4509
2000    4268
        ... 
1903       2
1915       1
1908       1
1917       1
1919       1
Name: year, Length: 106, dtype: int64

In [32]:
movie.loc[movie['year'] == '1995']

,movieId,title,genres,userId,rating,timestamp,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,1995
1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,1995
2,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,1995
3,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,1995
4,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,1995
...,...,...,...,...,...,...,...
95334,99130,Ice Cream Man,Comedy|Horror,89,3.5,1520409565,1995
98718,132084,Let It Be Me,(no genres listed),599,2.5,1519142705,1995
99071,136297,Mortal Kombat: The Journey Begins,Action|Animation,599,0.5,1519175186,1995
100280,167772,The Spirit of Christmas,Animation|Comedy,89,1.5,1520408904,1995


In [33]:
movie.loc[movie['genres'] == '(no genres listed)', 'genres'] = 'Unknown'

In [34]:
movie['genres'].value_counts()

Comedy                                       7196
Drama                                        6290
Comedy|Romance                               3967
Comedy|Drama|Romance                         3000
Comedy|Drama                                 2851
                                             ... 
Action|Animation|Children|Comedy|Musical        1
Adventure|Crime|Drama|Thriller                  1
Action|Adventure|Comedy|Drama|Romance|War       1
Comedy|Crime|Sci-Fi                             1
Action|Animation|Crime|Drama                    1
Name: genres, Length: 951, dtype: int64

In [35]:
movie.head()

,movieId,title,genres,userId,rating,timestamp,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,1995
1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,1995
2,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,1995
3,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,1995
4,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,1995


In [37]:
dummies = movie['genres'].str.get_dummies(sep='|')
movie = pd.concat([movie, dummies], axis=1)
movie.head()

,movieId,title,genres,userId,rating,timestamp,year,Action,Adventure,Animation,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,Unknown,War,Western
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,1995,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,1995,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,1995,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,1995,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,1995,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [38]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100806 entries, 0 to 100835
Data columns (total 27 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   movieId      100806 non-null  int64  
 1   title        100806 non-null  object 
 2   genres       100806 non-null  object 
 3   userId       100806 non-null  int64  
 4   rating       100806 non-null  float64
 5   timestamp    100806 non-null  int64  
 6   year         100806 non-null  object 
 7   Action       100806 non-null  int64  
 8   Adventure    100806 non-null  int64  
 9   Animation    100806 non-null  int64  
 10  Children     100806 non-null  int64  
 11  Comedy       100806 non-null  int64  
 12  Crime        100806 non-null  int64  
 13  Documentary  100806 non-null  int64  
 14  Drama        100806 non-null  int64  
 15  Fantasy      100806 non-null  int64  
 16  Film-Noir    100806 non-null  int64  
 17  Horror       100806 non-null  int64  
 18  IMAX         100806 non-

In [39]:
movie['year'] = movie['year'].astype(int)

In [40]:
movie['year'].dtype

dtype('int64')

In [41]:
movie['rating'].value_counts()

4.0    26814
3.0    20042
5.0    13205
3.5    13133
4.5     8547
2.0     7549
2.5     5548
1.0     2809
1.5     1791
0.5     1368
Name: rating, dtype: int64

In [44]:
movie['timestamp'].nunique()

85014